<a href="https://colab.research.google.com/github/rckormos/PyCCCP/blob/main/CCCP_Generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Coiled Coil Generation via the Crick Parameterization

User-friendly tools for generating coiled coil protein structures using the Crick parameterization as described in [Grigoryan and DeGrado, 2011](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3052747). For more details and the source code, please refer to the accompanying [GitHub repository](https://github.com/rckormos/PyCCCP).

In [1]:
#@title Input Crick parameters below, then hit `Runtime` -> `Run all`
from sys import version_info 
python_version = f"{version_info.major}.{version_info.minor}"

#@markdown  - For any of the parameter values below, semicolons can be used to 
#@markdown    separate multiple options. Coils will be generated for every 
#@markdown    combination of parameters. For example, if r0 is set to 
#@markdown    "4.0; 5.0; 6.0" and r1 is set to "2.2; 2.3", bundles for all six 
#@markdown    possible parameter combinations will be generated. This also 
#@markdown    holds for parameters with multiple values, in which case 
#@markdown    comma-separated parameter lists should themselves be separated by 
#@markdown    semicolons to ensure generation of structures using each of the 
#@markdown    different possible lists.

#@markdown ###<ins>**Global Parameters**</ins>
#@markdown **Number of Chains**
cN = "4" #@param {type:"string"}
#@markdown **Chain Length(s)**
chL = "36" #@param {type:"string"}
#@markdown  - If one chain length is provided, it will be applied to all chains. 
#@markdown    Otherwise, a length for each chain should be provided, separated 
#@markdown    by commas (e.g. "30, 31, 31, 30").

#@markdown **Superhelical Radius, $r_0$ (Ångstroms)**
r0 = "7.902;8.002;8.102;8.202" #@param {type:"string"}
#@markdown **Minor Helical Radius, $r_1$ (Ångstroms)**
r1 = "2.250" #@param {type:"string"}
#@markdown **Superhelical Frequency, $\omega_0$ (Degrees / Residue)**
w0 = "-3.114" #@param {type:"string"}
#@markdown **Minor Helical Frequency, $\omega_1$ (Degrees / Residue)**
w1 = "102.325" #@param {type:"string"}
#@markdown **Superhelical Pitch Angle, $\alpha$ (Degrees)**
a = "-16.626" #@param {type:"string"}

#@markdown ###<ins>**Symmetry Options**</ins>
#@markdown **Coil Symmetry**
symmetry = 'None' #@param ['None', 'Cn', 'D2n'] {type:"string"}
#@markdown - If this option is not 'None', the per-chain parameters below **will 
#@markdown   be overridden** to enforce the desired symmetry.

#@markdown ###<ins>**Per-Chain Parameters**</ins>
#@markdown **Minor Helical Phase Offset(s), $\phi_1$ (Degrees)**
ph1 = "-60.432,-50.045,-70.284,-52.207" #@param {type:"string"}
#@markdown  - If one phase angle is provided, it will be applied to all chains. 
#@markdown    Otherwise, an angle for each chain should be provided, separated 
#@markdown    by commas (e.g. "15, 30, 45, 60").

#@markdown **Chain Orientation**
cr = "ap,p,ap" #@param {type:"string"}
#@markdown  - One fewer orientation should be provided than there are chains. 
#@markdown    Parallel chains to the first chain should be represented as "p" 
#@markdown    and antiparallel chains should be represented as "ap". For 
#@markdown    example, a four-helix bundle with chains alternating between 
#@markdown    parallel and antiparallel should be given as "ap, p, ap" since the 
#@markdown    first chain is parallel to itself and thus should not be given.

#@markdown **Superhelical Relative Phase Offsets, $\Delta \phi_0$ (Degrees)**
dph0 = "-76.093,164.337,88;-76.093,166.391,90.054;-76.093,168.446,92.109;-76.093,170.5,94.163;-76.093,172.554,96.217;-76.093,174.608,98.271;-76.093,176.663,100.326;-76.093,178.717,102.38" #@param {type:"string"}
#@markdown  - One fewer relative phase offset should be provided than there are 
#@markdown    chains, since no information is lost if the first chain has a 
#@markdown    superhelical phase offset of zero.

#@markdown **Superhelical Relative z-Offsets, $\Delta z_{off}$ (Ångstroms)**
zoff = "4.045,0.673,4.027" #@param {type:"string"}
#@markdown  - One fewer relative z-offset should be provided than there are 
#@markdown    chains, since no information is lost if the first chain has a 
#@markdown    z-offset of zero. The type of z-offset is specified by the 
#@markdown    "ztype" field below.

#@markdown **Additional Residues**
dt = "None" #@param {type:"string"}
#@markdown  - The additional number of residues that will be added to each 
#@markdown    terminus of each chain, provided as a comma-separated list.  If 
#@markdown    None is given, then no additional residues will be added.

#@markdown ###<ins>**Miscellaneous Parameters**</ins>
#@markdown **z-Offset Type**
ztype = 'zoffaa' #@param ['default', 'apNNZoff', 'registerzoff', 'zoffaa'] {type:"string"}
#@markdown - 'default' -- The z-offset is interpreted as being between the 
#@markdown   N termini of chains 0 and k in all circumstances.
#@markdown - 'apNNzoff' -- The z-offset is interpreted as being between the 
#@markdown   N termini of chains 0 and k if k runs parallel to 0, and the N 
#@markdown   terminus of chain 0 and the C terminus of chain k if k is 
#@markdown   anti-parallel to chain 0.
#@markdown - 'registerzoff' -- The z-offset refers to the offset between points 
#@markdown   on the Crick curves of opposing chains that point directly into 
#@markdown   the interface.
#@markdown - 'zoffaa' -- The z-offset is interpreted as the offset between 'a' 
#@markdown   positions on opposing chains.

#@markdown **Backbone Type**
bbtype = 'ala' #@param ['ca', 'gly', 'ala'] {type:"string"}

#@markdown **Job Name**
jobname = 'combs_backbones' #@param {type:"string"}
#@markdown - This parameter will be used to generate output file names along 
#@markdown   with the job hash.

In [2]:
#@title Install dependencies
%%bash -s $python_version

set -e

PYTHON_VERSION=$1

if [ ! -f PYCCCP_READY ]; then
  echo "installing pycccp..."
  # install dependencies
  pip install -q "pycccp @ git+https://github.com/rckormos/PyCCCP"
  touch PYCCCP_READY
fi

installing pycccp...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 7.2 MB/s eta 0:00:00


In [ ]:
#@title Generate coils
from itertools import product

import numpy as np

from pycccp.PDBIO import coordsToPDB
from pycccp.generateCrickBB import generateCrickBB

def get_vals(instr, delimiter0=';', delimiter1='', outtype=str, 
             casting_dict={}):
  if len(delimiter1) and delimiter1 in instr:
    if len(casting_dict):
      vals = [[casting_dict[val.strip()] for val in 
               entry.strip().split(delimiter1)] 
               for entry in instr.split(delimiter0)]
    else:
      vals = [[outtype(val.strip()) for val in entry.strip().split(delimiter1)] 
               for entry in instr.split(delimiter0)]
  else:
    if len(casting_dict):
      vals = [casting_dict[entry.strip()] for entry in instr.split(delimiter0)]
    else:
      vals = [outtype(entry.strip()) for entry in instr.split(delimiter0)]
  return vals

cN_vals = get_vals(cN, outtype=int)
chL_vals = get_vals(chL, delimiter1=',', outtype=int)
r0_vals = get_vals(r0, outtype=float)
r1_vals = get_vals(r1, outtype=float)
w0_vals = get_vals(w0, outtype=float)
w1_vals = get_vals(w1, outtype=float)
a_vals = get_vals(a, outtype=float)
ph1_vals = get_vals(ph1, delimiter1=',', outtype=float)
cr_vals = get_vals(cr, delimiter1=',', casting_dict={'p' : 1, 'ap' : 0})
dph0_vals = get_vals(dph0, delimiter1=',', outtype=float)
zoff_vals = get_vals(zoff, delimiter1=',', outtype=float)
if dt != 'None':
  dt_vals = get_vals(dt, delimiter1=',', outtype=float)
else:
  dt_vals = [None]

# Handle symmetry
if symmetry == 'Cn':
  for i, ent_list in enumerate(ph1_vals):
    ph1_vals[i] = [ent_list[0]] * len(ent_list)
  for i, ent_list in enumerate(cr_vals):
    cr_vals[i] = [1] * len(ent_list)
  for i, ent_list in enumerate(dph0_vals):
    dph0_vals[i] = [360. * j // (len(ent_list) + 1) for j in 
                    range(1, len(ent_list) + 1)]
  for i, ent_list in enumerate(zoff_vals):
    zoff_vals[i] = [0.] * len(ent_list)
elif symmetry == 'D2n':
  for i, ent_list in enumerate(ph1_vals):
    assert len(ent_list) % 2 == 0
    ph1_vals[i] = [ent_list[0]] * len(ent_list)
  for i, ent_list in enumerate(cr_vals):
    assert len(ent_list) % 2 == 1
    cr_vals[i] = [0] + [1, 0] * (len(ent_list) // 2)
  for i, ent_list in enumerate(dph0_vals):
    assert len(ent_list) % 2 == 1
    dph0_vals[i] = [360. * j // (len(ent_list) + 1) for j in 
                    range(1, len(ent_list) + 1)]
  for i, ent_list in enumerate(zoff_vals):
    assert len(ent_list) % 2 == 1
    zoff_vals[i] = [0.] * len(ent_list)

params = product(cN_vals, chL_vals, r0_vals, r1_vals, w0_vals, w1_vals, a_vals, 
                 ph1_vals, cr_vals, dph0_vals, zoff_vals, dt_vals)
param_names = ['cN', 'chL', 'r0', 'r1', 'w0', 'w1', 'a', 'ph1', 'cr', 'dph0', 
               'zoff', 'dt', 'ztype', 'bbtype']

pdb_paths = []
counter = 0
for tup in params:
  _cN, _chL, _r0, _r1, _w0, _w1, _a, _ph1, _cr, _dph0, _zoff, _dt = tup
  all_params = [repr(val).replace("'", '') for val in 
                list(tup) + [ztype, bbtype]]
  XYZ, chains = generateCrickBB(_cN, _chL, _r0, _r1, _w0, _w1, _a, _ph1, _cr, 
                                _dph0, _zoff, _dt, ztype, bbtype)
  remarks = [name + ' = ' + param for name, param in 
             zip(param_names, all_params)]
  pdb_path = jobname + str(counter) + '.pdb'
  pdb_paths.append(pdb_path)
  coordsToPDB(XYZ, chains, pdb_path, bbtype)
  with open(pdb_path, 'a') as f:
    for i, remark in enumerate(remarks):
      f.write('REMARK 420 ' + remark)
      if i != len(remarks) - 1:
        f.write('\n')
  counter += 1

In [ ]:
#@title Visualize coils
import py3Dmol

view = py3Dmol.view(width=400, height=300)

for pdb_path in pdb_paths:
  with open(pdb_path) as ifile:
    system = "".join([x for x in ifile])
  view.addModelsAsFrames(system)

view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}, 
                              "stick": {'color': 'spectrum'}})
view.zoomTo()
view.show()

In [ ]:
#@title Package and download results
import os
import shutil
from google.colab import files

if not os.path.exists(f'{jobname}_result'):
  os.mkdir(f'{jobname}_result')

for path in os.listdir():
  if 'pdb' in path:
    shutil.move(path, f'{jobname}_result/' + path)
shutil.make_archive(f'{jobname}.result', 'zip', f'{jobname}_result')
files.download(f'{jobname}.result.zip')